<a href="https://colab.research.google.com/github/bartek-wojcik/usa_election_2020_sentiment_analysis/blob/master/sentiment_analysis_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#### Colab Setup

In [ ]:
# Install java
import os
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

# Install pyspark
! pip install --ignore-installed -q pyspark==2.4.4

# Install Spark NLP
! pip install --ignore-installed -q spark-nlp==2.6.5


openjdk version "1.8.0_275"
OpenJDK Runtime Environment (build 1.8.0_275-8u275-b01-0ubuntu1~18.04-b01)
OpenJDK 64-Bit Server VM (build 25.275-b01, mixed mode)


#### Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Sentiment Analysis Pipeline



#### 1. Call necessary imports

In [ ]:
#Imports
import sys

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.sql.functions import array_contains
from pyspark.ml import Pipeline, PipelineModel

from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline

#### 2. Load SparkSession

In [ ]:
import sparknlp
from pyspark.sql import SQLContext

session = SparkSession.builder \
    .appName("Spark NLP")\
    .master("local[4]")\
    .config("spark.driver.memory","8G")\
    .config("spark.executor.memory", "8G")\
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.11:2.6.5")\
    .config("spark.kryoserializer.buffer.max", "1000M")\
    .getOrCreate()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", session.version)

sc = session.sparkContext
sqlContext = SQLContext(sc)

Spark NLP version:  2.6.5
Apache Spark version:  2.4.4


#### 3. Load data

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

candidate = 'donaldtrump'

data = session.read\
        .option("sep", ",")\
        .option("multiLine", "true")\
        .option("header", "true")\
        .option("escape", '"')\
        .option("delimiter", ",")\
        .csv(f'/content/drive/MyDrive/Datasets/hashtag_{candidate}.csv')

data.show()

+-------------------+--------------------+--------------------+-----+-------------+-------------------+--------------------+--------------------+----------------+--------------------+-------------------+--------------------+--------------------+------------------+-------------------+-----------------+--------------------+-------------+--------------------+----------+--------------------+
|         created_at|            tweet_id|               tweet|likes|retweet_count|             source|             user_id|           user_name|user_screen_name|    user_description|     user_join_date|user_followers_count|       user_location|               lat|               long|             city|             country|    continent|               state|state_code|        collected_at|
+-------------------+--------------------+--------------------+-----+-------------+-------------------+--------------------+--------------------+----------------+--------------------+-------------------+---------------

In [ ]:
data.columns

['created_at',
 'tweet_id',
 'tweet',
 'likes',
 'retweet_count',
 'source',
 'user_id',
 'user_name',
 'user_screen_name',
 'user_description',
 'user_join_date',
 'user_followers_count',
 'user_location',
 'lat',
 'long',
 'city',
 'country',
 'continent',
 'state',
 'state_code',
 'collected_at']

#### 4. Clean data


In [ ]:
def prepare_df(df):
    df = df.drop('user_description')
    df = df.filter(df.tweet.isNotNull())
    df = df.filter(df.collected_at.isNotNull())
    df = df.withColumn('tweet', regexp_replace('tweet', 'http\S+', ""))
    df = df.withColumn('tweet', regexp_replace('tweet', '(@[A-Za-z]+[A-Za-z0-9-_]+)', ""))
    df = df.withColumn('tweet', regexp_replace('tweet', '[^a-zA-Z0-9 -]', ""))
    df = df.withColumn('tweet', regexp_replace('tweet', '/\s\s+/g', " "))
    df = df.withColumnRenamed('tweet', 'text')
    return df

data = prepare_df(data)

data.show()


+-------------------+--------------------+--------------------+-----+-------------+-------------------+--------------------+--------------------+----------------+-------------------+--------------------+--------------------+------------------+-------------------+-----------------+--------------------+-------------+--------------------+----------+--------------------+
|         created_at|            tweet_id|                text|likes|retweet_count|             source|             user_id|           user_name|user_screen_name|     user_join_date|user_followers_count|       user_location|               lat|               long|             city|             country|    continent|               state|state_code|        collected_at|
+-------------------+--------------------+--------------------+-----+-------------+-------------------+--------------------+--------------------+----------------+-------------------+--------------------+--------------------+------------------+-----------------

#### 5. Create pipelines


In [ ]:
ml_pipeline = PretrainedPipeline('analyze_sentiment', lang='en')
ml_pipeline.model.stages

analyze_sentiment download started this may take some time.
Approx size to download 4.9 MB
[OK!]


[document_b52f0e78aa1c,
 SENTENCE_199413a74375,
 REGEX_TOKENIZER_b6ca4d4dc972,
 SPELL_e4ea67180337,
 VIVEKN_8ac4c5e76848]

In [ ]:
%%time
ml_pipeline.annotate("I like to play football")

CPU times: user 25 ms, sys: 8.72 ms, total: 33.7 ms
Wall time: 60.9 ms


{'checked': ['I', 'like', 'to', 'play', 'football'],
 'document': ['I like to play football'],
 'sentence': ['I like to play football'],
 'sentiment': ['positive'],
 'token': ['I', 'like', 'to', 'play', 'football']}

In [ ]:
def ml_analyze(df):
  result = ml_pipeline.transform(df)
  return result

ml_result = ml_analyze(data)


In [ ]:
import pyspark.sql.functions as F

def results(df):
    return df.withColumn("sentiment_res", df.sentiment.getItem(0).result)

In [ ]:
%%time

results(ml_result).withColumn('sentiment_res', regexp_replace('sentiment_res', 'na', 'neutral'))\
    .select(['created_at', 'tweet_id', 'text', 'likes', 'retweet_count', 'source', 'user_join_date', 'user_followers_count', 'user_location', 'lat', 'long', 'city', 'country', 'continent', 'state', 'state_code', 'collected_at', 'sentiment_res'])\
    .toPandas().to_csv(f'/content/drive/MyDrive/Datasets/{candidate}_sentiment_ml.csv', header=True, index=False)

CPU times: user 27.7 s, sys: 2.19 s, total: 29.9 s
Wall time: 19min 32s
